In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

#### Do directors have an impact on success(nbox)?
#### Do actors have an impact on success(nbox)?

In [2]:
wikidata = pd.read_json('movies/data/wikidata-movies2.json.gz', orient='records', lines=True)
actors = pd.read_json('movies/data/actors.json.gz', orient='records', lines=True)
directors = pd.read_json('movies/data/directors.json.gz', orient='records', lines=True)

In [3]:
wikidata = wikidata[pd.notnull(wikidata['nbox'])]

In [4]:
# explode wikidata director
wikidata_d_stack = wikidata['director'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame()
wikidata_d_stack.columns = ['wikidata_id']
# explode wikidata actors
wikidata_a_stack = wikidata['cast_member'].apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame()
wikidata_a_stack.columns = ['wikidata_id']

In [5]:
# merge with label&wikidata_id DF
wikidata_d = wikidata_d_stack.reset_index().merge(directors, how="left", on="wikidata_id").set_index('index')
wikidata_a = wikidata_a_stack.reset_index().merge(actors, how="left", on="wikidata_id").set_index('index')

In [6]:
# merge back to wikidata
wikidata_d = pd.merge(wikidata_d, wikidata, left_index=True, right_index=True).reset_index()
wikidata_a = pd.merge(wikidata_a, wikidata, left_index=True, right_index=True).reset_index()
#df_genre = pd.merge(df_genre, df, left_index=True, right_index=True).reset_index()

#### AVONA test on directors and nbox

In [7]:
groups_d = wikidata_d.groupby('director_label').groups

In [8]:
# get n box values for each actor
array = []
for value in groups_d.values():
    array.append(wikidata_d['nbox'].iloc[value])

In [9]:
anova_d = stats.f_oneway(*array)

In [10]:
anova_d.pvalue
# 7.94 e-05 => there is a difference in means of the actors

7.947026173570023e-05

#### AVONA with actors

In [11]:
groups_actor = wikidata_a.groupby('actor_label').groups

In [12]:
# get n box values for each actor
array = []
for value in groups_actor.values():
    array.append(wikidata_a['nbox'].iloc[value])

In [13]:
anova_a = stats.f_oneway(*array)

In [15]:
anova_a.pvalue
# 0.0002 => actors nboxes means are different

0.00026458942201597086